# Django REST Framework: Nested Serializer Example

In this example, we demonstrate how to nest one serializer inside another — useful for one-to-many relationships like `Singer` and `Song`.

In [ ]:
# Optional Django setup (not required inside real project)
import os
import django
from pathlib import Path

BASE_DIR = Path.cwd() / "drf_nested_demo"
os.makedirs(BASE_DIR, exist_ok=True)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "drf_nested_demo.settings")

try:
    django.setup()
except:
    pass

## Step 1: Define the Models (`models.py`)

In [ ]:
# models.py
from django.db import models

class Singer(models.Model):
    name = models.CharField(max_length=100)
    gender = models.CharField(max_length=10)

    def __str__(self):
        return self.name

class Song(models.Model):
    title = models.CharField(max_length=100)
    duration = models.FloatField()
    singer = models.ForeignKey(Singer, related_name="songs", on_delete=models.CASCADE)

    def __str__(self):
        return self.title

## Step 2: Create Nested Serializers (`serializers.py`)

In [ ]:
# serializers.py
from rest_framework import serializers
from .models import Singer, Song

class SongSerializer(serializers.ModelSerializer):
    class Meta:
        model = Song
        fields = ['id', 'title', 'duration']

class SingerSerializer(serializers.ModelSerializer):
    songs = SongSerializer(many=True, read_only=True)

    class Meta:
        model = Singer
        fields = ['id', 'name', 'gender', 'songs']

## Step 3: Create ViewSets and Register Routes (`views.py`, `urls.py`)

In [ ]:
# views.py
from rest_framework import viewsets
from .models import Singer, Song
from .serializers import SingerSerializer, SongSerializer

class SingerViewSet(viewsets.ModelViewSet):
    queryset = Singer.objects.all()
    serializer_class = SingerSerializer

class SongViewSet(viewsets.ModelViewSet):
    queryset = Song.objects.all()
    serializer_class = SongSerializer

In [ ]:
# urls.py
from django.urls import include, path
from rest_framework.routers import DefaultRouter
from .views import SingerViewSet, SongViewSet

router = DefaultRouter()
router.register(r'singers', SingerViewSet)
router.register(r'songs', SongViewSet)

urlpatterns = [
    path('', include(router.urls)),
]

## Step 4: Example Output (GET `/singers/1/`)

```json
{
  "id": 1,
  "name": "A. R. Rahman",
  "gender": "Male",
  "songs": [
    {
      "id": 1,
      "title": "Jai Ho",
      "duration": 4.5
    },
    {
      "id": 2,
      "title": "Kun Faya Kun",
      "duration": 5.0
    }
  ]
}
```

## 🔍 Summary Table

| Element | Purpose |
|---------|---------|
| `SongSerializer` | Handles individual song representation |
| `SingerSerializer` | Embeds multiple `SongSerializer` via nested relation |
| `related_name="songs"` | Enables reverse lookup (e.g., `singer.songs`) |
| `many=True` | Indicates a list of related objects (one-to-many) |